# Backend 와 Frontend 어플리케이션 Build

이전 노트북에서는 강력한 에이전트/봇을 만드는 데 필요한 모든 기능과 코드를 모았습니다. 사용자의 질문에 따라 이 에이전트/봇은 사용 가능한 소스 및 도구에서 답변을 검색합니다.

그러나 다음과 같은 질문이 생깁니다.: **"다중 채널 배포를 지원할 수 있는 Bot 애플리케이션에 이 코드를 어떻게 통합할 수 있습니까?"** 이상적인 시나리오는 봇을 한 번 구축하고 MS Teams, Web Chat, Slack, Alexa, Outlook, WhatsApp, Line, Facebook 등과 같은 다양한 채널에 배포하는 것입니다.

이를 달성하려면 봇을 API로 구축하는 데 도움을 줄 뿐만 아니라 이 API를 여러 채널에 쉽게 노출할 수 있는 서비스가 필요합니다. 이 서비스를 <u>Azure Bot Framework</u>라고 합니다.

본 노트북에서는 Bot Framework API와 Service를 이용하여 지금까지 Bot API로 개발한 코드를 배포하는 방법을 알아봅니다.

## Azure Bot Framework와 Bot Service란?

Microsoft Bot Framework 및 Azure Bot Service는 지능형 봇을 구축, 테스트, 배포 및 관리할 수 있는 라이브러리, 도구 및 서비스 모음입니다.

봇은 웹 애플리케이션으로 구현되는 경우가 많으며 Azure에서 호스팅되고 API를 사용하여 메시지를 보내고 받습니다.

Azure Bot Service 및 Bot Framework에는 다음이 포함됩니다.

- C#, JavaScript, Python 또는 Java로 봇을 개발하기 위한 Bot Framework SDK입니다.
- 엔드투엔드 봇 개발에 도움이 되는 CLI 도구입니다.
- 봇과 채널 간에 메시지와 이벤트를 중계하는 봇 커넥터 서비스.
- 봇 관리 및 구성을 위한 Azure 리소스입니다.

따라서 애플리케이션을 구축하기 위해 **Bot Framework Python SDK를 사용하여 웹 API를 구축**하고 **Bot Service를 사용하여 API를 여러 채널에 연결**합니다.

## Architecture

아래 이미지는 다음을 보여줍니다.
1) Bot API를 호스팅하는 Azure 웹앱
2) Bot API, 채널 및 Application Insights 간의 연결을 제공하는 Azure Bot Service

![Botframework](./images/Bot-Framework.png)

# Backend - Azure Web App - Bot API

Brain Agent를 생성하기 위해 이전 노트북에서 사용한 모든 기능과 프롬프트는 각각 `utils.py`와 `prompts.py`에 있습니다.
따라서 수행해야 할 작업은 기본적으로 이전 Notebook에서 수행한 것과 동일한 작업을 Bot Framework Python SDK 클래스 내에서 수행하는 것입니다.

`apps/backend/`폴더에는 `app.py`, `bot.py`, `config.py`라는 세 가지 파일이 있습니다..
- `app.py`: 애플리케이션의 진입 메인 포인트입니다.
- `bot.py`: OpenAI 관련 코드가 있는 곳입니다.
- `config.py`: API가 수신할 포트와 App Service 주체 변수 이름을 선언합니다.

`bot.py`만 처리하면 됩니다. 여기에는 Azure OpenAI 애플리케이션과 관련된 모든 논리 코드가 있습니다.

`apps/backend/README.md` 에서 다음 방법에 대한 모든 지침을 찾을 수 있습니다.
1) Azure 웹 서비스 배포: Azure Web App 및 Azure Bot Service
2) 코드를 압축하여 Azure 웹앱에 업로드합니다.
3) Azure Portal에서 Bot Service를 사용하여 Bot API를 테스트합니다.

지금 바로 `apps/backend/README.md`의 지침을 따르세요.

# Frontend - Azure Web App - Streamlit 

Backend Bot API 앱이 실행되고 Bot Service Azure Portal을 사용하여 성공적으로 테스트되면 이제 샘플 UI 빌드를 진행할 수 있습니다.

`apps/frontend/` 폴더에는 다음을 포함하는 간단한 Streamlit 애플리케이션을 구축하는 데 필요한 파일이 있습니다.

1) <u>A Search Interface</u>: `utils.py`, `prompts.py` 및 스트림라이트 기능 사용
2) <u>A WebChat Interface</u>: Bot Service 웹 채팅 자바스크립트 라이브러리를 사용하여 Streamlit 내에서 웹챗 채널을 html 구성 요소로 렌더링할 수 있습니다.

(1)에서는 논리 코드가 Frontend Web App에서 실행되고 있지만 (2)에서는 논리 코드가 Backend Bot API에서 실행되고 있으며 Frontend는 Bot Service의 WebChat 채널을 사용하고 있습니다.

지금 바로 `apps/frontend/README.md`의 지침을 따르세요.

# GitHub Actions로 빌드 및 배포

모든 구성 요소를 수동으로 빌드하고 배포하지 않으려면 `.github/workflows/main_gptsmartsearch_apps.yml`에 GitHub 자동화 CI 파이프라인이 제공됩니다. 
CI 파이프라인 설계에 대한 몇 가지 참고 사항::
- "branch per environment approach"를 사용합니다. 배포 환경 이름은 "set environment for branch" 단계(line 29)의 분기/환경 이름 매핑 논리를 기반으로 'runtime'에 계산 됩니다. 현재 구현된 로직은 모든 ​​것을 개발 환경에 매핑합니다. 따라서 `main branch`의 각 git push에서 파이프라인은 `Development`이라는 환경에 배포하려고 시도하면서 트리거됩니다. GitHub 환경과 특정 환경 변수 및 비밀을 설정하는 방법에 대한 자세한 내용은 [here](https://docs.github.com/en/actions/deployment/targeting-different-environments/using-environments-for-deployment)를 참조하세요.
- GitHub 환경 변수 및 비밀은 개발 환경별 구성을 구성하는 데 사용됩니다. Github 저장소 설정에서 수동으로 구성해야 합니다.
    - `secrets.AZUREAPPSERVICE_X_PUBLISHPROFILE`은 Azure 앱 서비스 게시 프로필 구성을 안전하게 저장하는 데 사용됩니다.
    - `vars.AZURE_WEBAPP_X_NAME`은 Infra Arm 배포 중에 생성된 Azure 웹 앱 리소스 이름을 저장하는 데 사용됩니다.
- PAzure 웹앱이 현재 SCM_DO_BUILD_DURING_DEPLOYMENT로 구성되어 있으므로 빌드 단계에서는 Python 종속성 설치가 비활성화됩니다. 빌드 작업 중에 종속성 해결을 활성화하는 데 사용할 수 있는 환경 속성이 있습니다. `DO_BUILD_DURING_DEPLOYMENT : false `를 설정하면 됩니다.

백엔드 및 프런트엔드 구성 요소 모두에 대해 자동화된 빌드 및 배포를 올바르게 구성하려면 다음 단계를 따르세요.
 
 1. GitHub의 포크된 저장소로 이동하여 'Development'라는 [environment]((https://docs.github.com/en/actions/deployment/targeting-different-environments/using-environments-for-deployment))을 만듭니다. (정확한 이름입니다. 변경하지 마세요). 환경 이름을 변경하고 새 분기 및 환경을 추가하거나 현재 분기/환경 매핑을 변경하려면 그렇게 할 수 있지만 `.github/workflows/main_gptsmartsearch_apps.yml` (starting line 29)에서 그에 따라 파이프라인 코드를 변경해야 합니다.
 2. 프런트엔드 및 백엔드 Azure 웹 앱 [publish profiles]((https://learn.microsoft.com/en-us/visualstudio/azure/how-to-get-publish-profile-from-azure-app-service?view=vs-2022)) 모두에 대한 'Development' environment [secrets](https://docs.github.com/en/actions/security-guides/encrypted-secrets#creating-encrypted-secrets-for-a-repository)을 만듭니다. .PublishSettings 파일의 xml 콘텐츠를 복사하여 비밀 값에 붙여넣어야 합니다. :
     - 이름이 `AZUREAPPSERVICE_BACKEND_PUBLISHPROFILE`인 비밀을 만들고 `apps/backend/README.md`에서 'Deploy to Azure' 버튼을 사용하여 만든 Azure 웹 앱의 프로필을 게시하도록 값 필드를 설정합니다.
     - 이름이 `AZUREAPPSERVICE_FRONTEND_PUBLISHPROFILE`인 비밀을 만들고 `apps/frontend/README.md`에서 'Deploy to Azure' 버튼을 사용하여 만든 Azure 웹 앱의 프로필을 게시하도록 값 필드를 설정합니다.
3. 프런트엔드 및 백엔드 Azure 웹 앱 리소스 이름 모두에 대한 'Development' environment [variables](https://docs.github.com/en/actions/learn-github-actions/variables#creating-configuration-variables-for-an-environment)를 만듭니다. :
    - `AZURE_WEBAPP_BACKEND_NAME` 이름의 변수를 생성하고 `apps/backend/README.md`에서 'Deploy to Azure' 버튼을 사용하여 생성한 Azure 웹 앱 리소스 이름으로 값 필드를 설정합니다.
 - `AZURE_WEBAPP_FRONTEND_NAME` 이름의 변수를 생성하고 `apps/frontend/README.md`에서 'Deploy to Azure' 버튼을 사용하여 생성한 Azure 웹 앱 리소스 이름으로 값 필드를 설정합니다.
4. 각 커밋에 대해 GitHub Actions 탭에서 트리거된 파이프라인의 상태를 확인하면 특정 커밋에 대해 파이프라인이 트리거된 것을 확인할 수 있습니다. 모든 것이 정상이면 아래와 같이 파이프라인 세부 정보의 빌드 및 배포 작업 모두에 녹색 확인 표시가 표시됩니다.:

![pipeline success](./images/github-actions-pipeline-success.png)


# Reference

- https://learn.microsoft.com/en-us/azure/bot-service/bot-service-overview?view=azure-bot-service-4.0
- https://github.com/microsoft/botbuilder-python/tree/main
- https://github.com/microsoft/BotFramework-WebChat/tree/master